# Training

This notebook demonstrates the process of training an ensemble learning model using a provided CSV file. It showcases data preprocessing, model training, evaluation, and saving the trained model. The ensemble method (hard voting, soft voting, or stacking) can be selected based on the user's choice.


## Importing Necessary Libraries

First, we import all the necessary libraries and modules needed for this script. This includes libraries for handling warnings, data manipulation, machine learning, and the custom Ensemble module containing ensemble learning methods.

In [1]:
import pandas as pd
import os
import numpy as np

import sklearn
from sklearn.naive_bayes import BernoulliNB
# from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt 
from src import Utils, Bayes

/tmp/ipykernel_25093/831435522.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Constants

In [2]:
TEST_SIZE = 0.2

MODEL_FOLDER = 'models/model_bayes' # Where models will be saved
DATASET = 'datasets/datasetall.csv'


## Utility Functions

### Model saving folder creation

Create a folder to save the model based on `MODEL_FOLDER`

In [3]:
os.makedirs(
  MODEL_FOLDER,
  exist_ok=True, # Create folder if it doesn't exist, else do nothing
)

print(f"Saving model at '{MODEL_FOLDER}'")

Saving model at 'models/model_bayes'


### Function to Read CSV File

The `read_csv_file` function reads the CSV file and returns a pandas DataFrame. If the file is not found, the script will exit with an error message.

In [4]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', usecols=range(2))
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

dataset = read_csv_file(DATASET)
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [5]:
dataset['label'].value_counts(ascending=True)

label
0    14115
1    14346
Name: count, dtype: int64

### Function for Train-Test Split

The `get_train_test_split` function splits the dataset into training and testing sets with an 80-20 split ratio and returns them.

In [6]:
def shuffle_data_frame(data_frame):
    text = list(data_frame['text'])
    label = list(data_frame['label'])

    assert(len(text) == len(label))

    indices = list(range(len(label)))

    # Make a random number generator that will shuffle list of indices
    # It is seeded to be reproducible
    random_number_generator = np.random.default_rng(seed=0)
    random_number_generator.shuffle(indices)

    shuffled_text = []
    shuffled_labels = []

    # Iterate through the list of indices and add the original data
    # from those shuffled indices
    for index in indices:
        shuffled_text.append(text[index])
        shuffled_labels.append(label[index])

    return pd.DataFrame({
        'text': shuffled_text,
        'label': shuffled_labels,
    })


def get_train_test_split(data_frame: pd.DataFrame, test_size: float):
    """
    Makes a stratified train test split.
    This aims to preserve the distribution between classes.
    """
    if not (1 > test_size > 0):
        print('ERROR: test_size must be between 0 and 1')
        return

    data_frame = shuffle_data_frame(data_frame)

    data_frame_length = len(data_frame)
    train_size = 1 - test_size

    nonhate_rows = data_frame[data_frame['label'] == 0] 
    nonhate_row_length = len(nonhate_rows)

    nonhate_row_train_size = math.ceil(nonhate_row_length * train_size)

    nonhate_row_train = nonhate_rows[0:nonhate_row_train_size]
    nonhate_row_test = nonhate_rows[nonhate_row_train_size:nonhate_row_length]

    assert(len(nonhate_row_train) + len(nonhate_row_test) == nonhate_row_length)

    hate_rows = data_frame[data_frame['label'] == 1] 
    hate_row_length = len(hate_rows)

    hate_row_train_size = math.ceil(hate_row_length * train_size)

    hate_row_train = hate_rows[0:hate_row_train_size]
    hate_row_test = hate_rows[hate_row_train_size:hate_row_length]

    assert(len(hate_row_train) + len(hate_row_test) == hate_row_length)

    combined_train = pd.concat([nonhate_row_train, hate_row_train])
    combined_test = pd.concat([nonhate_row_test, hate_row_test])

    shuffled_train = shuffle_data_frame(combined_train)
    shuffled_test = shuffle_data_frame(combined_test)

    return (
        shuffled_train['text'],
        shuffled_test['text'],
        shuffled_train['label'],
        shuffled_test['label'],
    )

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

## Train Data

In [7]:
pd.DataFrame(X_train)

,text
0,[USERNAME] Palangga ka man sang mga taga Baco...
1,Who dafuq is Jose Montemayor Jr.???
2,Di na nakakatuwa yung mukha ni Mar Roxas sa TV...
3,national elections. | via[USERNAME]
4,"Binay will be staring in a movie called ""The D..."
...,...
22764,"""Kala ko wala andito pala si Marcos.""*pertaini..."
22765,sie ~ [USERNAME]Marcos Magnanakaw Marcos Dikta...
22766,If Mar is BatMarBinay is Bane-ay.
22767,to my moots im sorry in not sorry for flooding...


In [8]:
y_train_dataframe = pd.DataFrame(y_train, columns=['label'])
y_train_dataframe

,label
0,0
1,0
2,1
3,0
4,1
...,...
22764,0
22765,1
22766,1
22767,1


In [9]:
y_train_dataframe.value_counts(ascending=True)

label
0        11292
1        11477
Name: count, dtype: int64

## Test Data

In [10]:
pd.DataFrame(X_test)

,text
0,Bakit trending ang Only Binay?
1,Mare @ Cebu [USERNAME][USERNAME] Marcos Never ...
2,Kahit anong gawin ko bakit di ko ma appreciate...
3,Oras na para tayo'y bumoto ng taong mag tataas...
4,VP[USERNAME]is currently in Zamboanga Sibugay ...
...,...
5687,[USERNAME] Laban LeniAngat Buhay LahatLeni Kiko
5688,Nagconcede ka man Maimarwala ka prinnagdala ka...
5689,Did You Know that former Philippine secretary ...
5690,Bakit nakakairita commercial ni Mar Roxas?


In [11]:
y_test_dataframe = pd.DataFrame(y_test, columns=['label'])
y_test_dataframe

,label
0,0
1,1
2,1
3,0
4,0
...,...
5687,0
5688,1
5689,0
5690,1


In [12]:
y_test_dataframe.value_counts(ascending=True)

label
0        2823
1        2869
Name: count, dtype: int64

## CountVectorizer

In [13]:
CountVec = sklearn.feature_extraction.text.CountVectorizer()

In [ ]:
tokenizer = CountVec.build_tokenizer()

In [ ]:
X_train_tokenized = [
  tokenizer(text)
  for text in X_train
]

In [ ]:
X_train_tokenized

In [ ]:
pd.DataFrame(X_train_tokenized[:20])

In [14]:
X_train_transformed = CountVec.fit_transform(
  X_train
  # ' '.join(X_train_tokenized)
)

In [15]:
sklearn.feature_extraction.text.tokenized_text

[['username',
  'palangga',
  'ka',
  'man',
  'sang',
  'mga',
  'taga',
  'bacolod',
  'vp',
  'username',
  'let',
  'leni',
  'lead',
  'monlaurista',
  'username',
  'grabeh',
  'ang',
  'loilo',
  'palangga',
  'ka',
  'namon',
  'username',
  'leni',
  'kiko',
  'leni',
  'robredo',
  'let',
  'leni',
  'lead',
  'iloilo',
  'kakampinklaban',
  'leni'],
 ['who', 'dafuq', 'is', 'jose', 'montemayor', 'jr'],
 ['di',
  'na',
  'nakakatuwa',
  'yung',
  'mukha',
  'ni',
  'mar',
  'roxas',
  'sa',
  'tv',
  'nakakatamad',
  'manood'],
 ['national', 'elections', 'via', 'username'],
 ['binay',
  'will',
  'be',
  'staring',
  'in',
  'movie',
  'called',
  'the',
  'dark',
  'force',
  'awakens',
  'username',
  'nog',
  'nog',
  'hambog'],
 ['the',
  'fact',
  'that',
  'we',
  'are',
  'even',
  'discussing',
  'the',
  'probability',
  'of',
  'binaymarcosduterte',
  'administration',
  'is',
  'horrifying'],
 ['username', 'lito', 'atienza', 'na', 'goi'],
 ['yung',
  'mga',
  'yaman

In [ ]:
X_test_transformed = CountVec.transform(X_test)

In [ ]:
pd.DataFrame([CountVec.vocabulary_]).T.head(30)

In [ ]:
print(X_train_transformed)

## Bernoulli Naive Bayes

In [ ]:
train_bayes = Bayes.BayesPipeline

train_bayes

In [ ]:
train_bayes.fit(X_train, y_train)

In [ ]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_val,
  y_val,
  train_bayes,
)

In [ ]:
test_accuracy, test_recall, test_precision, test_f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_bayes,
)

In [ ]:
Utils.save_trained_model(train_bayes, f"{MODEL_FOLDER}/Bayes")